# Installation


# Data Prep

In [114]:
# prompt: install tensorflow, pennylane, and all the related libraries that may be needed to use those two libraries.

!pip install autograd jax jaxlib
!pip install tf-nightly
!pip install git+https://github.com/PennyLaneAI/pennylane.git
!pip install pennylane-lightning pennylane-lightning[GPU]
!pip install tensorflow-datasets

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/PennyLaneAI/pennylane.git to /tmp/pip-req-build-l4q5_6oc
  Running command git clone --filter=blob:none --quiet https://github.com/PennyLaneAI/pennylane.git /tmp/pip-req-build-l4q5_6oc
  Resolved https://github.com/PennyLaneAI/pennylane.git to commit 9685bd4699cd707b543186e9eda82b959618fcda
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 23.3.1 -> 23.3.2


In [ ]:
pip show tensorflow

In [ ]:
# prompt: import tensorflow and pennylane
import tensorflow as tf
# import pennylane as qml
import numpy as np

In [100]:
SEED = 10
BATCH_SIZE = 1024
BUFFER_SIZE = 10000

## Using the Dataset

### Loading from TSDS

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
train = tfds.load('squad/v2.0', split='train', shuffle_files=True)
dev = tfds.load('squad/v2.0', split='validation', shuffle_files=True)

In [ ]:
trainTopics = ['Computer', 'IPod', 'Germans', 'Qing_dynasty', 'YouTube', 'Airport', 'Russian_language', 'Armenians', 'Matter']
devTopics = ['Computational_complexity_theory', 'Steam_engine', 'Normans']

In [58]:
train.element_spec

{'answers': {'answer_start': TensorSpec(shape=(None,), dtype=tf.int32, name=None),
  'text': TensorSpec(shape=(None,), dtype=tf.string, name=None)},
 'context': TensorSpec(shape=(), dtype=tf.string, name=None),
 'id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'is_impossible': TensorSpec(shape=(), dtype=tf.bool, name=None),
 'plausible_answers': {'answer_start': TensorSpec(shape=(None,), dtype=tf.int32, name=None),
  'text': TensorSpec(shape=(None,), dtype=tf.string, name=None)},
 'question': TensorSpec(shape=(), dtype=tf.string, name=None),
 'title': TensorSpec(shape=(), dtype=tf.string, name=None)}

In [59]:
def filterTopics(data, topics, field):
    wanted = []
    refTrain = train.enumerate(start=0).as_numpy_iterator()
    
    for x in refTrain:
        title = x[1]['title'].decode()
        
        if (x[1]['is_impossible'] == False) & (title in topics):
            query = x[1][field].decode()

            wanted.append(query)

    return wanted

def getTrain(ds):
    filteredContext = filterTopics(ds, trainTopics, 'context')
    print(len(filteredContext))
    filteredQuest = filterTopics(ds, trainTopics, 'question')
    print(len(filteredQuest))

    
    return tf.data.Dataset.from_tensor_slices(filteredContext), tf.data.Dataset.from_tensor_slices(filteredQuest)

def getY(ds):
    ds = filterTopics(ds, devTopics)
    ds = ds.filter(lambda x: x['is_impossible'] == False)
    return ds.map(lambda x: [x['answers']['answer_start'], x['answers']['answer_start']+len(x['answers']['text'])])

context, quest = getTrain(train)
# trainY = train.apply(getY)

context


2024-01-30 05:46:05.310148: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1590
question
In what year, was the first Nobel Prize in Physics won?
Which publication praised iPods for revolutionizing the industry?
What group took over the British Airports Authority in 2006?
HP-branded iPods accounted for what share of iPod purchases?
What did Pushkin reject from older Russian?
What are three games, in addition to Brick, which have been included with the iPod?
What part of the USSR did Armenia join?
How are 'dental' sounds pronounced?
The astrolabe was a combination of what two devices in history?
What percentage of 14 year olds in 2009 said they would choose German for their nationality if they got to choose?
Youtube offers users the option to watch content where?
What happened to the Old Summer Palace?
By what time period had many of the Jewish  population joined the Holy Roman Empire?
Who made Beijing his capital?
What kind of security determines if files moved to a computer can be played?
Devices that give input or output to a computer are called what?
What a

2024-01-30 05:46:45.611138: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [60]:
context.take(1).get_single_element(), quest.take(1).get_single_element()

(<tf.Tensor: shape=(), dtype=string, numpy=b'The work of David Hilbert and Max Planck was crucial to the foundation of modern physics, which Werner Heisenberg and Erwin Schr\xc3\xb6dinger developed further. They were preceded by such key physicists as Hermann von Helmholtz, Joseph von Fraunhofer, and Gabriel Daniel Fahrenheit, among others. Wilhelm Conrad R\xc3\xb6ntgen discovered X-rays, an accomplishment that made him the first winner of the Nobel Prize in Physics in 1901. The Walhalla temple for "laudable and distinguished Germans", features a number of scientists, and is located east of Regensburg, in Bavaria.'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'In what year, was the first Nobel Prize in Physics won?'>)

## Vectorizing Datasets

### Counting Vocabs

In [85]:
contextIter = context.enumerate(start=0)
questIter = quest.enumerate(start=0)

def countSeq(data):
    allWords = []
    maxSeq = 0

    for entry in data:
        words = entry[1].decode().split(" ")
    
        if len(words) > maxSeq:
          maxSeq = len(words)
    
        allWords.extend(words)

    print(allWords[:10])
    
    vocabs = len(set(allWords))
    
    return vocabs, maxSeq

contextVocabs, contextMaxSeq = countSeq(contextIter.as_numpy_iterator())
questVocabs, questMaxSeq = countSeq(questIter.as_numpy_iterator())

['The', 'work', 'of', 'David', 'Hilbert', 'and', 'Max', 'Planck', 'was', 'crucial']


2024-01-30 06:05:02.902636: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


['In', 'what', 'year,', 'was', 'the', 'first', 'Nobel', 'Prize', 'in', 'Physics']


2024-01-30 06:05:03.161589: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [86]:
contextVocabs, contextMaxSeq, questVocabs, questMaxSeq

(10789, 407, 3532, 23)

### Vectorizing all Sentences and Words

In [68]:
# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
context_vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=contextVocabs,
    output_mode='int',
    output_sequence_length=contextMaxSeq)

quest_vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=questVocabs,
    output_mode='int',
    output_sequence_length=questMaxSeq)

In [87]:
context_vectorize_layer.adapt(context.batch(BATCH_SIZE))
quest_vectorize_layer.adapt(quest, context.batch(BATCH_SIZE))

2024-01-30 06:07:54.583583: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2024-01-30 06:08:01.213754: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [88]:
contextVectVocabs = context_vectorize_layer.get_vocabulary()
questtVectVocabs = quest_vectorize_layer.get_vocabulary()

In [89]:
contextVect = context.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).map(context_vectorize_layer).unbatch()
questVect = quest.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE).map(quest_vectorize_layer).unbatch()

### Successfully Vectorized Sequences of Words stored as contextSeq and questSeq

In [90]:
contextSeq = list(contextVect.as_numpy_iterator())
len(contextSeq)

2024-01-30 06:08:17.123155: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1590

In [91]:
questSeq = list(questVect.as_numpy_iterator())
len(questSeq)

2024-01-30 06:08:20.779137: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


1590

In [92]:
for seq in contextSeq[:5]:
  print(f"{seq} => {[contextVectVocabs[i] for i in seq]}")

[   2  191    3 5454 5098    4 2008 4628    9 5471    6    2 2037    3
  100 2454   21 1961 5111    4 5298 4420  208  708   50   16 2444   10
   44  665 4637    8 5105  433 5109 1450  433 5212    4 5197 5456 5260
  254  258 1348 5516 4435 1321 4017   19 5849   14  126  518    2   42
 4041    3    2 2488 2435    5 2454    5 2090    2 4068 4230   12 4911
    4 3299   94  479    7  133    3 4416    4   15  856  145    3 4508
    5 1112    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 


### Function to utilize Skip-gram Seq2Vect grouping for Training data formatting

In [95]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.

import tqdm

def skipGram(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, neighbors, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, neighbor_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([neighbor_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      neighbors.append(context)
      labels.append(label)

  return targets, neighbors, labels

In [96]:
contextTargets, contextNeighbors, contextLabels = skipGram(contextSeq, window_size=2, num_ns=4, vocab_size=contextVocabs, seed=SEED)
questTargets, questNeighbors, questLabels = skipGram(questSeq, window_size=2, num_ns=4, vocab_size=questVocabs, seed=SEED)

100%|██████████| 1590/1590 [00:02<00:00, 696.54it/s]


In [99]:
contextTargets = np.array(contextTargets)
contextNeighbors = np.array(contextNeighbors)
contextLabels = np.array(contextLabels)

print('\n')
print(f"ContextTargets.shape: {contextTargets.shape}")
print(f"ContextNeighbors.shape: {contextNeighbors.shape}")
print(f"ContextLabels.shape: {contextLabels.shape}")



ContextTargets.shape: (143882,)
ContextNeighbors.shape: (143882, 5)
ContextLabels.shape: (143882, 5)


In [103]:
questTargets = np.array(questTargets)
questNeighbors = np.array(questNeighbors)
questLabels = np.array(questLabels)

print('\n')
print(f"QuestTargets.shape: {questTargets.shape}")
print(f"QuestNeighbors.shape: {questNeighbors.shape}")
print(f"QuestLabels.shape: {questLabels.shape}")



QuestTargets.shape: (5625,)
QuestNeighbors.shape: (5625, 5)
QuestLabels.shape: (5625, 5)


In [105]:
trainable = tf.data.Dataset.from_tensor_slices(((contextTargets, contextNeighbors), contextLabels))
trainable = trainable.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
trainable


<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


In [119]:
trainable = trainable.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
trainable


<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>

### Just Load Previous rawContext, rawQuest, contextVect, questVect, trainable

In [121]:
loadedRawContext = tf.data.Dataset.load("RawContext")
loadedRawQuest = tf.data.Dataset.load("RawQuest")
loadedContextVect = tf.data.Dataset.load("ContextVect")
loadedQuestVect = tf.data.Dataset.load("QuestVect")
loadedTrainable = tf.data.Dataset.load("Trainable")

loadedRawContext, loadedRawQuest, loadedContextVect, loadedQuestVect, loadedTrainable

(<_LoadDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>,
 <_LoadDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>,
 <_LoadDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int64, name=None)>,
 <_LoadDataset element_spec=TensorSpec(shape=(None,), dtype=tf.int64, name=None)>,
 <_LoadDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>)

# Model

## Classical

In [ ]:
class Classical(tf.keras.layers.Layer):
  def __init__(self, num_outputs):
    super(Classical, self).__init__()
    self.num_outputs = num_outputs

  def build(self, input_shape):
    self.kernel = self.add_weight("kernel", shape=[int(input_shape[-1]), self.num_outputs])

  def call(self, inputs):
    return tf.matmul(inputs, self.kernel)

layer = MyDenseLayer(10)

In [ ]:

dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):
    # ...
